In [ ]:
import numpy as np
#from oneibl.one import ONE
from one.api import ONE
import brainbox.io.one as bbone

from math import *
import sys
import scipy.stats as scist
from os import path

import pandas as pd

from scipy.stats import rankdata



######



one = ONE()
one.refresh_cache('refresh')

from brainbox.population.decode import get_spike_counts_in_bins

from brainbox.io.one import SpikeSortingLoader

from ibllib.atlas import AllenAtlas

ba = AllenAtlas()


from brainbox.task.closed_loop import generate_pseudo_blocks

In [ ]:
from iblutil.numerical import ismember

In [ ]:
from brainwidemap import bwm_query

from pathlib import Path
from brainbox.io.one import SessionLoader

from brainwidemap import bwm_units, bwm_query, load_good_units, load_trials_and_mask, filter_sessions, \
    download_aggregate_tables

# Specify a path to download the cluster and trials tables
local_path = Path.home().joinpath('bwm_examples')
local_path.mkdir(exist_ok=True)

In [ ]:
# download table of QC-good units 
bwm_2024_df = bwm_units(one,  min_units_sessions=(5, 2))

In [ ]:
# list of pid
pid_2024=bwm_2024_df['pid'].unique()

In [ ]:
# spike_rate: PETH aligned to  stimulus, spike_rate=(number of neurons, number of time-bin[-0.2:0.01:0.3], number of condtions)
#3 conditions: spike_rate[:,:,0]: contrast Left trials, 
#              spike_rate[:,:,1]: contrast Zero trials,
#              spike_rate[:,:,2]: contrast Right trials,
# spike_rate_2: PETH aligned to  choice  spike_rate_2=(number of neurons, number of time-bin[-0.2:0.01:0.3], number of condtions)
#3 conditions: spike_rate_1[:,:,0]: contrast Left trials, 
#              spike_rate_2[:,:,1]: contrast Zero trials,
#              spike_rate_3[:,:,2]: contrast Right trials,
# spike_rate_3: PETH aligned to reward   spike_rate_3=(number of neurons, number of time-bin[-0.2:0.01:0.3], number of condtions)
#3 conditions: spike_rate_1[:,:,0]: contrast Left trials, 
#              spike_rate_2[:,:,1]: contrast Zero trials,
#              spike_rate_3[:,:,2]: contrast Right trials,



def BWM_PETH(pid, eid, TimeWindow=np.array([-0.2, 0.3]), BinSize=0.01):


     
    temp_spikes, temp_clusters = load_good_units(one, pid, compute_metrics=False)

    clusters = temp_clusters[temp_clusters['uuids'].isin(list(units_df.uuids))]
    spike_idx, ib = ismember(temp_spikes['clusters'], clusters.index)
    clusters.reset_index(drop=True, inplace=True)

    spikes = {k: v[spike_idx] for k, v in temp_spikes.items()}
    spikes['clusters'] = clusters.index[ib].astype(np.int32)
    
    
    
    
    #######################
    
    trials, mask = load_trials_and_mask(one, eid, min_rt=0.08, max_rt=2., nan_exclude='default')
    # select good trials 
    trials=trials.loc[mask == True]

    stim_on=trials.stimOn_times.to_numpy()
    response=trials.response_times.to_numpy()
    feedback=trials.feedback_times.to_numpy()
    contrast_R=trials.contrastRight.to_numpy()
    contrast_L=trials.contrastLeft.to_numpy()
    choice=trials.choice.to_numpy()
    block=trials.probabilityLeft.to_numpy()
    first_move=trials.firstMovement_times.to_numpy()
    
    #######################
    
    
    
    

                 


    num_neuron=len(np.unique(spikes['clusters']))
    num_trial =len(stim_on)
    num_cond=3




    # compute correct choice of trial (include correct trials or trials with zero contrast)
    left_stim_index=np.argwhere(contrast_L>0)
    right_stim_index=np.argwhere(contrast_R>0)
    zero_contrast_index=np.argwhere(np.logical_or(contrast_R==0,contrast_L==0))

    trial_answer=np.zeros(len(choice))
    trial_answer[left_stim_index[:,0]]=1
    trial_answer[right_stim_index[:,0]]=-1
    
    
    
    
    num_bin=np.floor((TimeWindow[1]-TimeWindow[0])/BinSize).astype(int)
    
    # spike_rate: PETH aligned to  stimulus 
    # spike_rate_2: PETH aligned to  choice  
    # spike_rate_3: PETH aligned to reward
    
    spike_rate=np.zeros((num_neuron,num_bin,num_cond))
    spike_rate_2=np.zeros((num_neuron,num_bin,num_cond))
    spike_rate_3=np.zeros((num_neuron,num_bin,num_cond))
    
    


    for i_cond in range(num_cond): 
        
        if i_cond==0:
            temp_stim_on=stim_on[left_stim_index[:,0]]
            temp_first_move=first_move[left_stim_index[:,0]]
            temp_feedback=feedback[left_stim_index[:,0]]
            
        elif i_cond==1:
            temp_stim_on=stim_on[zero_contrast_index[:,0]]
            temp_first_move=first_move[zero_contrast_index[:,0]]
            temp_feedback=feedback[zero_contrast_index[:,0]]
            
        elif i_cond==2:
            temp_stim_on=stim_on[right_stim_index[:,0]]
            temp_first_move=first_move[right_stim_index[:,0]]
            temp_feedback=feedback[right_stim_index[:,0]]
        
        
        
        
        for i_bin in range(num_bin):
    
            T_1= TimeWindow[0]+i_bin*BinSize
            T_2= TimeWindow[0]+(i_bin+1)*BinSize

            raw_events=np.array([temp_stim_on+T_1, temp_stim_on+T_2]).T
            events=raw_events
            
            
            raw_events_2=np.array([temp_first_move+T_1, temp_first_move+T_2]).T
            events_2=raw_events_2
            
            
            raw_events_3=np.array([temp_feedback+T_1, temp_feedback+T_2]).T
            events_3=raw_events_3

        
            spike_count, cluster_id = get_spike_counts_in_bins(spikes['times'],spikes['clusters'],events)

            

            spike_rate[:,i_bin,i_cond]=   np.nanmean(spike_count,axis=1)/BinSize

        
        
            spike_count_2, cluster_id = get_spike_counts_in_bins(spikes['times'],spikes['clusters'],events_2)
            
            spike_rate_2[:,i_bin,i_cond]=  np.nanmean(spike_count_2,axis=1)/BinSize
        

            spike_count_3, cluster_id = get_spike_counts_in_bins(spikes['times'],spikes['clusters'],events_3)

            
            spike_rate_3[:,i_bin,i_cond]=  np.nanmean(spike_count_3,axis=1)/BinSize
        

   
    ############ return allen region id  ########################
    area_label=clusters['atlas_id'][cluster_id].to_numpy()

    ############ return cluster id ########################
    QC_cluster_id=clusters['cluster_id'][cluster_id].to_numpy()
   
           
        

    return spike_rate, spike_rate_2, spike_rate_3, area_label, QC_cluster_id

##############################################################################

In [ ]:
########### compute PETH for the entire dataset ###########
num_cond=3
num_bin=50

ses_area_label=np.zeros(0)

ses_rate_1=np.zeros((0,num_bin,num_cond))
ses_rate_2=np.zeros((0,num_bin,num_cond))
ses_rate_3=np.zeros((0,num_bin,num_cond))

session_id=np.zeros(0)
ses_QC_cluster_id=np.zeros(0)


num_session=len(pid_2024)

for i_ses in range(num_session):
#for i in ses_list:
    
    #i_ses=ses_list[i]
    
    pid=pid_2024[i_ses]
    
    temp_list=bwm_2024_df.loc[bwm_2024_df['pid'] == pid_2024[i_ses]]
    eid=temp_list['eid'].to_numpy()[0]
    
    

    print(i_ses)
    print(pid)
            

         
    rate_1, rate_2, rate_3, area_label, QC_cluster_id = BWM_PETH(pid, eid)   
    ses_rate_1=np.append(ses_rate_1,rate_1,axis=0)
    ses_rate_2=np.append(ses_rate_2,rate_2,axis=0)
    ses_rate_3=np.append(ses_rate_3,rate_3,axis=0)

    
    ses_area_label=np.append(ses_area_label,area_label)
    ses_area_label=ses_area_label.astype(int) 
                            
    session_id=np.append(session_id,i_ses*np.ones(len(area_label)))
    session_id=session_id.astype(int)
    ses_QC_cluster_id=np.append(ses_QC_cluster_id,QC_cluster_id)
    ses_QC_cluster_id=ses_QC_cluster_id.astype(int) 

In [ ]:
np.save('BWM_2024_PETH_stim',ses_rate_1)
np.save('BWM_2024_PETH_move',ses_rate_2)
np.save('BWM_2024_PETH_feedback',ses_rate_3)

np.save('BWM_2024_PETH_allen_id',ses_area_label)
np.save('BWM_2024_PETH_session_id',session_id)